## ipyrad tutorial notebook
#### UNAM 4/20/2018


In [3]:
import ipyrad

In [4]:
# step 1 of an ipyrad analysis (ipyrad -n name)
data = ipyrad.Assembly("name")

New Assembly: name


In [17]:
# step 2 is to set the parameters
data.set_params("project_dir", "test")
data.set_params("raw_fastq_path", "./ipsimdata/rad_example_R1_.fastq.gz")
data.set_params("barcodes_path", "./ipsimdata/rad_example_barcodes.txt")
data.get_params()

0   assembly_name               name                                         
1   project_dir                 ./test                                       
2   raw_fastq_path              ./ipsimdata/rad_example_R1_.fastq.gz         
3   barcodes_path               ./ipsimdata/rad_example_barcodes.txt         
4   sorted_fastq_path                                                        
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    rad                                          
8   restriction_overhang        ('TGCAG', '')                                
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6                               

In [18]:
# run steps of our analysis. Run step 1
data.run("1")

Assembly: name
[####################] 100%  sorting reads         | 0:00:03 | s1 | 
[####################] 100%  writing/compressing   | 0:00:01 | s1 | 


In [19]:
#

,state,reads_raw
1A_0,1,19862
1B_0,1,20043
1C_0,1,20136
1D_0,1,19966
2E_0,1,20017
2F_0,1,19933
2G_0,1,20030
2H_0,1,20199
3I_0,1,19885
3J_0,1,19822


In [20]:
data.run("234567")

Assembly: name
[####################] 100%  processing reads      | 0:00:03 | s2 | 
[####################] 100%  dereplicating         | 0:00:00 | s3 | 
[####################] 100%  clustering            | 0:00:01 | s3 | 
[####################] 100%  building clusters     | 0:00:00 | s3 | 
[####################] 100%  chunking              | 0:00:00 | s3 | 
[####################] 100%  aligning              | 0:00:10 | s3 | 
[####################] 100%  concatenating         | 0:00:00 | s3 | 
[####################] 100%  inferring [H, E]      | 0:00:03 | s4 | 
[####################] 100%  calculating depths    | 0:00:00 | s5 | 
[####################] 100%  chunking clusters     | 0:00:00 | s5 | 
[####################] 100%  consens calling       | 0:00:13 | s5 | 
[####################] 100%  concat/shuffle input  | 0:00:00 | s6 | 
[####################] 100%  clustering across     | 0:00:01 | s6 | 
[####################] 100%  building clusters     | 0:00:00 | s6 | 
[##################

In [28]:
# create three separate assemblies for each clade
data1 = data.branch('data1', [name for name in data.samples if "1" in name])
data2 = data.branch('data2', [name for name in data.samples if "2" in name])
data3 = data.branch('data3', [name for name in data.samples if "3" in name])

In [33]:
keep2 = [i for i in data.samples if i not in keep]
keep2

['2G_0', '3K_0', '3J_0', '2E_0', '3I_0', '3L_0', '2F_0', '2H_0']

## Analyses

In [48]:
import ipyrad.analysis as ipa

In [60]:
rax = ipa.raxml(name="test", data=data.outfiles.phy)
rax.params.o = keep
rax.params.N = 200
rax.params

N        200                 
T        4                   
binary   raxmlHPC-PTHREADS-SSE3
f        a                   
m        GTRGAMMA            
n        test                
o        ['1A_0', '1B_0', '1C_0', '1D_0']
p        54321               
s        ~/test/name_outfiles/name.phy
w        ~/analysis-raxml    
x        12345               

In [62]:
rax.run(force=True)

job test finished successfully


In [63]:
import toytree
toytree.tree(rax.trees.bestTree).draw()

(<toyplot.canvas.Canvas at 0x7f1d71914e90>,
 <toyplot.coordinates.Cartesian at 0x7f1d718dcc50>)

1D_0 1C_0 1A_0 1B_0 3L_0 3K_0 3I_0 3J_0 2H_0 2G_0 2E_0 2F_0

## Prettier text

In [38]:
# haploid and diploid samples
dips = data.branch("diploids", keep)
haps = data.branch("haploids", keep2)

# set parameters for each assembly
dips.set_params("max_alleles_consens", 2)
haps.set_params("max_alleles_consens", 1)

# run both assemblies from step 4
dips.run("45", force=True)
haps.run("45", force=True)

Assembly: diploids
[####################] 100%  inferring [H, E]      | 0:00:01 | s4 | 
[####################] 100%  calculating depths    | 0:00:00 | s5 | 
[####################] 100%  chunking clusters     | 0:00:00 | s5 | 
[####################] 100%  consens calling       | 0:00:04 | s5 | 
Assembly: haploids
Applying haploid-based test (infer E with H fixed to 0)
[####################] 100%  inferring [H, E]      | 0:00:01 | s4 | 
[####################] 100%  calculating depths    | 0:00:00 | s5 | 
[####################] 100%  chunking clusters     | 0:00:00 | s5 | 
[####################] 100%  consens calling       | 0:00:10 | s5 | 


In [41]:
# this is the merged assembly of haploids and diploids
merged = ipyrad.merge("merged", [dips, haps])

In [43]:
merged.run("67")

Assembly: merged
[####################] 100%  concat/shuffle input  | 0:00:00 | s6 | 
[####################] 100%  clustering across     | 0:00:01 | s6 | 
[####################] 100%  building clusters     | 0:00:00 | s6 | 
[####################] 100%  aligning clusters     | 0:00:04 | s6 | 
[####################] 100%  database indels       | 0:00:00 | s6 | 
[####################] 100%  indexing clusters     | 0:00:00 | s6 | 
[####################] 100%  building database     | 0:00:00 | s6 | 
[####################] 100%  filtering loci        | 0:00:00 | s7 | 
[####################] 100%  building loci/stats   | 0:00:00 | s7 | 
[####################] 100%  building vcf file     | 0:00:01 | s7 | 
[####################] 100%  writing vcf file      | 0:00:00 | s7 | 
[####################] 100%  building arrays       | 0:00:00 | s7 | 
[####################] 100%  writing outfiles      | 0:00:00 | s7 | 
Outfiles written to: ~/test/merged_outfiles



In [26]:
keep = ['1A_0', '1B_0', '1C_0', '1D_0']
keep

['1A_0', '1B_0', '1C_0', '1D_0']

In [27]:
keep2 = [name for name in data.samples if "1" in name]
keep2

['1A_0', '1B_0', '1C_0', '1D_0']

In [24]:
clade1 = data.branch("clade1", keep)

In [25]:
clade1.stats

,state,reads_raw,reads_passed_filter,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
1A_0,6,19862,19862,1000,1000,0.001824,0.000759,1000
1B_0,6,20043,20043,1000,1000,0.001908,0.000752,1000
1C_0,6,20136,20136,1000,1000,0.002084,0.000745,1000
1D_0,6,19966,19966,1000,1000,0.001803,0.000761,1000
